#Energy Analysis: Willem Road 13

In [10]:
import numpy as np
import scipy as sp
import pandas as pd
import pyradi.ryplot as ryplot
import os.path
from scipy.optimize import curve_fit
import datetime

%matplotlib inline

# %reload_ext autoreload
# %autoreload 2

import pyradi.ryplot as ryplot
import pyradi.ryplanck as ryplanck
import pyradi.ryfiles as ryfiles
import pyradi.rymodtran as rymodtran

# import xlsxwriter

from IPython.display import HTML
from IPython.display import Image
from IPython.display import display
from IPython.display import FileLink, FileLinks

#make pngs at 150 dpi
import matplotlib as mpl
mpl.rc("savefig", dpi=150)
mpl.rc('figure', figsize=(10,8))
# %config InlineBackend.figure_format = 'svg'

pd.set_option('display.max_columns', 80)
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', 150)

In [37]:
filename = 'data/201506.csv'
df = pd.read_csv(filename)
df.columns = ['DateTime','Wm']
# df = df[pd.notnull(df['Wm'])]
df = df[df.Wm.notnull()]
df['kWh'] = df['Wm'] * 60. / 1000.
# df['Date'] = datetime.datetime.strptime(df['DateTime'], '%Y-%m-%d')
# df['Date'] = datetime.datetime.date(df['DateTime'])
print(df)
# print(type(df['DateTime']),type(df['Wm']))



                  DateTime    Wm     kWh
7830   2015-06-06 10:30:00  3048  182.88
7831   2015-06-06 10:31:00   674   40.44
7832   2015-06-06 10:32:00   541   32.46
7833   2015-06-06 10:33:00   448   26.88
7834   2015-06-06 10:34:00   362   21.72
7835   2015-06-06 10:35:00   366   21.96
7836   2015-06-06 10:36:00   355   21.30
7837   2015-06-06 10:37:00   359   21.54
7838   2015-06-06 10:38:00   360   21.60
7839   2015-06-06 10:39:00   375   22.50
7840   2015-06-06 10:40:00  2211  132.66
7841   2015-06-06 10:41:00  2248  134.88
7842   2015-06-06 10:42:00  2238  134.28
7843   2015-06-06 10:43:00  1848  110.88
7844   2015-06-06 10:44:00   357   21.42
7845   2015-06-06 10:45:00  1740  104.40
7846   2015-06-06 10:46:00  1395   83.70
7847   2015-06-06 10:47:00  1388   83.28
7848   2015-06-06 10:48:00  1621   97.26
7849   2015-06-06 10:49:00  1637   98.22
7850   2015-06-06 10:50:00  1626   97.56
7851   2015-06-06 10:51:00  1628   97.68
7852   2015-06-06 10:52:00  1599   95.94
7853   2015-06-0